In [1]:
from logp.zq.io.base import sparse_batch
import tensorflow as tf

from logp.io.elem_filter import filt_elems
import os, warnings
import pandas as pd
warnings.filterwarnings('ignore')

from logp.sf_behler.g3D import XYZ_dict
from rdkit import Chem
from logp.sf_behler.sf import sum_G1, sum_G2
from logp.sf_behler.sf import get_charge
from logp.zq.io.smi2feat import generator,generator_feat

from logp.zq.io.pdb2feat import generator_feat_md

from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
import multiprocessing as mp
from time import time

import numpy as np

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
np.set_printoptions(threshold=np.inf)

In [2]:
dataset = 'logp/DATASETS/MD/Martel/md_Martel_CHON.csv'

In [3]:
dataset = pd.read_csv(dataset)

In [4]:
smiles = dataset['smile'].values
labels = dataset['logp'].values

In [5]:
#labels

In [6]:
params = {'G1':{'eta':np.arange(1,1.8,0.1),'Rs':np.arange(0,0.8,0.1)},
          'G2':{'lamb':[1.0,-1.0],'eta':np.arange(1,1.8,0.1),'xi':np.arange(1,17.,2)},}

In [7]:
# Utilize the multiprocessing to speed the calculation

prefix = 'logp/DATASETS/MD/Martel/'
weights = None
max_out = True

pool = mp.Pool(mp.cpu_count())

dataset1 = []
#dataset2 = []
begin_time = time()
for i,smile in enumerate(smiles):
    #p= pool.apply_async(generator_feat_md,args=(prefix,smile,labels[i],params,weights,max_out))
    #dataset1.append(p)
    #dataset2.append(p_max)
    try:
    #    print(smile,labels[i])
        p = pool.apply_async(generator_feat_md,args=(prefix,smile,labels[i],params,weights,max_out))
        dataset1.append(p)
    #    dataset2.append(p_max)
    except:
        print('{} failed!'.format(smile))
pool.close()
pool.join()
end_time = time()
print("{} elasped".format(end_time-begin_time))

55841.362931489944 elasped


In [8]:
dataset_multi = {'elems':[],'coord':[],'fps':[],'logp':[]}
for r in dataset1:
    #print(r.get())
    dataset_multi['elems'].append(r.get()['elems'])
    dataset_multi['coord'].append(r.get()['coord'])
    dataset_multi['fps'].append(r.get()['fps'])
    dataset_multi['logp'].append(r.get()['logp'])

In [9]:
df = pd.DataFrame(data=dataset_multi)

In [10]:
df.to_csv('logp/DATASETS/sf_ave_Martel_MD_weights_None.csv',index=False)

In [11]:
dataset_multi_max = {'elems':[],'coord':[],'fps':[],'logp':[]}
for r in dataset1:
    #print(r.get())
    dataset_multi_max['elems'].append(r.get()['max_elems'])
    dataset_multi_max['coord'].append(r.get()['max_coord'])
    dataset_multi_max['fps'].append(r.get()['max_fps'])
    dataset_multi_max['logp'].append(r.get()['max_logp'])

In [12]:
df_max = pd.DataFrame(data=dataset_multi_max)

In [13]:
df_max.to_csv('logp/DATASETS/sf_max_Martel_MD_weights_None.csv',index=False)